# SERPENS
## An Introduction

In this notebook, I will demonstrate the basic usage of SERPENS.


***
***

### Using the Interface Script
This is the easiest way of using SERPENS and does not require any coding from you. However, it is limited in its scope. More advanced features of SERPENS are only accessible when you create your own script.

The first thing you are going to be faced with, is the setup of the planetary system. Already defined systems can be found in the _resources/objects.txt_ file and will be listed as you run the script. You may choose to add a new system by either editing that file directly or interacting with the interface script by answering the prompted question with "_n_" or "_no_". Editing the file directly allows you to define multiple moons as in the case of the Jupiter system.

Currently implemented exoplanet systems:
1. Jupiter
2. WASP-49
3. HD-189733
4. HD-209458
5. HAT-P-1
6. WASP-39
7. WASP-17
8. WASP-69
9. WASP-96
10. XO-2 N
11. WASP-52
etc.

Do you want to analyze an already set up exoplanet system? [Y/n]

If you answer above question with _n_, you will be guided through the setup of a new system. However, you will only be able to define a system with a star, planet and a single moon. 

In the following we will continue with the Jupiter system by entering the number 1 when prompted. 
A list of the currently defined parameters will be shown. These are also saved in file, namely _resources/input_parameters.txt_. I highly recommend changing any desired parameters directly inside the file, although you are able to change some basic parameters on the fly with the interface script.

After the parameter setup, the only thing left to do is to enter a savename and a save frequency. Latter refers to how often the simulation instances will be saved for later analysis. Entering, for example, _5_ will result in the simulation being saved every 5th advance. 

You are all set! The simulation will now start.

***
***

### Quick Start
Now, let's dig a little bit deeper and assume you want to work with SERPENS.

We want to start a simulation. We do so by initiating a _SerpensSimulation_ instance and advancing it an integer number of times. Inside the _Parameters_ class we have a variable called _sim_advance_. This variable describes the timestep of one advance in fractions of a full orbit (this is not the integration step time, but the step time after which new particles are added!).

The following advances 5 times, i.e. 5 _sim_advance_ orbit fractions.

In [ ]:
from serpens_simulation import SerpensSimulation

sersim = SerpensSimulation()
sersim.advance(5)

Once the simulation has run, we can run the analyzer to extract information from the particle distribution. Just like with the simulation we need to initialize a _SerpensAnalyzer_ instance.

We first plot a top-down view of the system and afterwards we take a look at the line-of-sight perspective.

In [3]:
%matplotlib inline
%matplotlib notebook
from serpens_analyzer import SerpensAnalyzer

serana = SerpensAnalyzer()
serana.top_down(timestep=5, d=2, colormesh=False, scatter=True, triplot=False, lim=15)


<IPython.core.display.Javascript object>

If you have set up the interactive framework for matplotlib correctly, you should be able to see plots (second after you close the first).

Now, let's look at another feature of SERPENS: The scheduler.

With the scheduler you are able to set up multiple simulations at once. These will get saved in separate folders. For each simulation the saved files are the parameters, a hash library and an archive bin-file. In order to analyze these, you need to move these files into the main folder of SERPENS afterwards. Note, however, that you need to analyze the simulations separately.

In order for the simulation to read the scheduler, we need to tell it if we deal with a moon or a planet (in case we want to simulate close orbiting evaporative planets).
The stars, planets and moons are set up in the _objects.py_ file. One needs to have the different systems and their celestial objects pre-defined. These celestial sets are indexed with the _celest_set_ attribute.
In addition to the system, we can define a list of species to check (here it's only sodium) with their respective parameters (see further down below).
If we want to change some parameters of the celestial objects, e.g. mass and radius, we can do so with the _objects_ argument. This argument must be given as a dictionary with the celestial bodies and their new parameters as a sub-dictionary. Valid arguments inside the sub-dictionary are REBOUND particle parameters and the argument _source_. Latter is a boolean that allows the user to set a different object as the simulation particle source. The original intention of this argument was to switch between sources if multiple natural satellites have been defined.


As always: We initiate the corresponding instance. As an example we want to simulate a super-Earth and an exo-Io in the systems WASP-49 and HD-189733, respectively.

Note: Units are in SI.


In [ ]:
from scheduler import SerpensScheduler
from species import Species

ssch = SerpensScheduler()

ssch.schedule("WASP-49-SuperEarth",
              species=[Species('Na', description='Na--125285kg/s--2.5-30km/s', n_th=0, n_sp=1000,
                       mass_per_sec=125285, lifetime=4*60, model_smyth_v_b=2500, model_smyth_v_M=30000)],
              moon=True,
              celest_set=2)
ssch.schedule("HD-189733-ExoIo",
              species=[Species('Na', description='Na--11576kg/s--2.5-30km/s', n_th=0, n_sp=1000, mass_per_sec=11576, lifetime=16.9 * 60, model_smyth_v_b=2500, model_smyth_v_M=30000)],
              moon=True,
              objects={'moon': {'m': 8.8e22, 'r': 1820e3}},
              celest_set=3)
ssch.run()

After the above code is finished you should find a new folder inside SERPENS with the resulting simulation instances.

Note: Running via the scheduler will always advance as often as has been defined by _num_sim_advances_ inside the _parameters_ class.

***
***

### Custom setup
For a basic simulation setup, we need to specify the following things:
- Define the celestial objects in the _objects.py_ file.
- Define the simulation parameters in the _init.py_ file.

***

#### Setting up parameters

The _init.py_ file contains the _DefaultFields_ class, which sets the general simulation parameters.
As the name suggests we are able to set the default parameters in this class.


Let us take a look at this class:

Running the _SerpensSimulation_ will use these parameters.
If you set up a scheduler you can overwrite the parameters for a single simulation. Once the simulation finished they will be reset to the default values. In case you prefer to work with the scheduler you can set up a simulation with new parameters in there.


Working with your own files, only importing SERPENS, you can also change the default parameters outside of the _parameters.py_ file using the _change_defaults_ method. You can pass your default parameters just like they are defined in the _DefaultFields_ class, i.e. the integration specifics as a dictionary, for example. These dictionaries need to contain *all* keys and values that you can see in the class and not only the parameters you want to change.

In [ ]:
from parameters import DefaultFields

DefaultFields.change_defaults(
    int_spec={
        "moon": True,
        "sim_advance": 1 / 80,
        "num_sim_advances": 100,
        "stop_at_steady_state": False,
        "gen_max": None,
        "r_max": 4,
        "random_walk": False,
        "particle_interpolation": False
    })


This does not make you completely independent of working inside the SERPENS files, though. The celestial objects are currently only definable inside the _objects.py_ file. Same goes for the species and chemical network.

#### Setting up celestial objects

The _objects.py_ file contains the _celestial_objects_ function, contain a dictionary of celestial objects.

We distinguish between systems containing a moon-source and system with a planet-source. If you want to change the celestial objects in a system, in which you want to simulate a moon,
make sure to modify the entries under the _if moon_ clause.
SERPENS allows to set default parameters for both cases.

Very important is to define which object is the source. In case of an exomoon system, you need to add _"hash": "moon"_ to the source's characteristic dictionary.
In all cases you need to add the _"hash": "planet"_ dictionary entry to the outgassing planet / exomoon primary.
If you analyze an exomoon system, but don't have an object with entry _"hash": "moon"_, SERPENS throws an error. It also throws an error if there's no object with _"hash": "planet"_.

ATTENTION: Even if you add additional moons/planets, only add the "moon"/"planet" hashes to ONE of those objects. SERPENS identifies the source and primary with these exact hashes.
Additional objects may carry any hash except "moon"/"planet".

You can also define multiple celestial object system, which is import if you want to schedule multiple SERPENS simulations in different exoplanet systems.
Make sure to define the dictionaries with _celest{X}_, where _{X}_ is a set number. This set number can later be put as an argument (see function arguments).

Example:

In [ ]:
def celestial_objects(moon, set=1):
    m_sol = 1.988e30
    m_jup = 1.898e27
    r_sol = 696340e3
    r_jup = 69911e3
    au = 149597870700

    if moon:
        # Jupiter
        # -------
        celest1 = {
            "star": {"m": 1.988e30,
                     "hash": 'star',
                     "r": 696340000
                     },
            "planet": {"m": 1.898e27,
                       "a": 7.785e11,
                       "e": 0.0489,
                       "inc": 0.0227,
                       "r": 69911000,
                       "primary": 'star',
                       "hash": 'planet'
                       },
            "Europa": {"m": 4.799e22,  # Europa
                     "a": 6.709e8,
                     "e": 0.009,
                     "inc": 0.0082,
                     "r": 1560800,
                     "primary": 'planet',
                     "hash": 'moon'
                     },
            "Io": {"m": 8.932e22,  # Io
                    "a": 4.217e8,
                    "e": 0.0041,
                    "r": 1821600,
                    "primary": 'planet'
                     }
        }
    else:
        # 55 Cnc-e
        # --------
        celest1 = {
            "star": {"m": 1.799e30,
                     "hash": 'star',
                     "r": 6.56e8
                     },
            "planet": {"m": 4.77179e25,
                       "a": 2.244e9,
                       "e": 0.05,
                       "inc": 0.00288,
                       "r": 1.196e7,
                       "primary": 'star',
                       "hash": 'planet'}
        }

In the above code we set two default systems.
If we set _int_spec["moon"] = True_ in the _Parameters_ class, then the Jovian system will be set as default. Note how only Europa has the hash "moon".
If we set _int_spec["moon"] = False_, then the 55 Cnc-e system will be set as default.


***

### Species

This section covers a quick explanation to the species class.

Species(
> name: str,
>> The name of the species. See the _species.py_ file for all implemented species names, e.g. "Na", "H2", "SO2".

> description: str,
>> If given, this description will be forwarded to any plots and allows a more sophisticated labeling, eg. "H2--6.69kg/s--1200m/s".

> n_th: int,
>> Number of thermal evaporated superparticles added to the simulation per advance.

> n_sp: int,
>> Number of sputtered superparticles added to the simulation per advance.

> mass_per_sec: float,
>> Mass flow rate in kg/s. Needed for density estimations.

> duplicate: int,
>> If multiple species of the same kind are analyzed, but with different parameters, this parameter must be increased by one for each duplicate species starting at 1.

> beta: float,
>> Radiation force to gravitational force ratio parameter.

> lifetime: float,
>> Lifetime of the species.

> n_e: float,
>> Electron density scaling factor. Will be passed to the chemical network, where all reaction rates given per electron density, will be scaled accordingly.

> sput_spec: dict
>> Contains all the parameters for the sputtering distribution. Most important are _model_smyth_v_b_ and _model_smyth_v_M_ which define most likely and maximum speeds, respectively.
)



#### Adding new species

If you want to add new species, there are a few steps you need to undertake:
1. Add species name to _implemented_species_ in the _Species_ class from the _species.py_ file and assign it a number acting as an id.
2. In _species.py_, there's a code block starting with _if name in self.implementedSpecies:_. Inside this block you need to make a new entry similar to what you can see already exists. The number passed to _super()_ is the mass number of the species.
3. Define a network or lifetime in the _network.py_ file that is associated to the new species id.